In [1]:
import pandas as pd

In [2]:
#Read excel files and create the dfs
hefzi_df = pd.read_excel('../Data/iCHOv1_Reconstruction.xlsx', header = 1)
fouladiha_df = pd.read_excel('../Data/10529_2020_3021_MOESM1_ESM.xlsx', 'Supplementary Table 10', header = 1)
yeo_df = pd.read_excel('../Data/Supplementary Data.xlsx', 'Data S2')

In [3]:
#Standarization of the columns names
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula'}, inplace = True)
yeo_df.rename(columns = {'Rxn':'Reaction', 'Subsystem (iCHO1766)':'Subsystem'}, inplace = True)

# Addition of a 'tag' column
hefzi_df.insert(loc=0, column='tag', value='hef')
fouladiha_df.insert(loc=0, column='tag', value='fou')
yeo_df.insert(loc=0, column='tag', value='yeo')


#fouladiha_df = fouladiha_df[['Reaction', 'Reaction Name', 'Subsystem', 'Reaction Formula', 'GPR']]

In [4]:
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+yeo_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

In [5]:
def add_col(df):
    '''
    This function adds the remaining columns from the cols list 
    to an existing df that are not contained in such df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [6]:
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
yeo_df = add_col(yeo_df)

In [7]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['tag', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References']]
yeo_df = yeo_df[['tag', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References']]
hefzi_df = hefzi_df[['tag', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References']]

In [8]:
# Merge all the df into a unified df
all_dfs = pd.concat([hefzi_df, fouladiha_df, yeo_df])
all_dfs = all_dfs.reset_index(drop = True)

In [9]:
# Group the entire dataset by reaction name 
all_dfs = all_dfs.groupby(['Reaction'], group_keys=True).apply(lambda x: x)

# Save merged datasets as Excel file
all_dfs.to_excel('../Data/all_dfs.xlsx')

In [ ]:
all_dfs